In [1]:
import os, sys
import geopandas as gpd
import pandas as pd
import rasterio as rio
import numpy as np
from shapely.geometry import Point
import skimage.graph as graph

In [2]:
sys.path.append('/home/wb514197/Repos/gostrocks/src') # gostrocks is used for some basic raster operations (clip and standardize)
sys.path.append('/home/wb514197/Repos/GOSTNets_Raster/src') # gostnets_raster has functions to work with friction surface
sys.path.append('/home/wb514197/Repos/GOSTnets') # it also depends on gostnets for some reason
sys.path.append('/home/wb514197/Repos/INFRA_SAP') # only used to save some raster results
# sys.path.append('/home/wb514197/Repos/HospitalAccessibility/src') # only used to save some raster results

In [3]:
import GOSTRocks.rasterMisc as rMisc
import GOSTNetsRaster.market_access as ma
from infrasap import aggregator

no xarray


In [4]:
iso3 = 'LBR'

In [5]:
input_dir = "/home/public/Data/PROJECTS/Health" #
out_folder = os.path.join(input_dir, "output", iso3)
if not os.path.exists(out_folder):
    os.mkdir(out_folder)

### Destinations

In [48]:
lbr_master = pd.read_csv(os.path.join(input_dir, "from_tashrik", "master lists", "Liberia MFL Adjusted.csv"), index_col=0)
lbr_geocoded = pd.read_csv(os.path.join(input_dir, "from_tashrik", "master lists", "Liberia MFL Adjusted Geocoded_Validated.csv"), index_col=0)

In [49]:
len(lbr_master)

873

In [50]:
lbr = lbr_master.loc[lbr_master.functional=="functional"].copy()

In [51]:
len(lbr)

872

In [37]:
lbr = lbr.loc[~(lbr.latitude.isna())].copy()

In [38]:
lbr.facilitytype.value_counts()

clinic           696
health center     86
hospital          24
health post        6
schoold based      4
SDP                1
Name: facilitytype, dtype: int64

In [39]:
len(lbr), len(lbr_geocoded)

(817, 55)

In [40]:
lbr_geocoded.geocoding_method.value_counts()

Location from old Master List     43
District name and county query     8
Location from Health Sites IO      2
Fuzzy match to OSM by Tashrik      2
Name: geocoding_method, dtype: int64

In [41]:
lbr_geocoded = lbr_geocoded.loc[lbr_geocoded.geocoding_method!="District name and county query"].copy()

In [47]:
55-47

8

In [42]:
len(lbr), len(lbr_geocoded)

(817, 47)

In [43]:
lbr = pd.concat([lbr, lbr_geocoded])

In [44]:
lbr.reset_index(drop=True, inplace=True)

In [45]:
lbr.columns

Index(['admin_county', 'admin_district', 'today', 'latitude', 'longitude',
       '_getcoordinate_altitude', 'precision', 'facilitytype', 'ownership',
       'opendate', 'functional', 'offer_anc', 'offer_obstetric', 'offer_emonc',
       'offer_postpartum', 'offer_newborncare', 'offer_maternalnewboen',
       'offer_csection', 'offer_childimmune', 'offer_imci',
       'offer_essentialnutrition', 'offer_fpcouple', 'offer_fpcounsel',
       'offer_adolescentsrhr', 'offer_sexgender', 'offer_detectmanage',
       'offer_precenttreat', 'dup', 'update_date', 'geocoding_method',
       'facility'],
      dtype='object')

In [31]:
lbr.facilitytype.value_counts()

clinic           741
health center     88
hospital          24
health post        6
schoold based      4
SDP                1
Name: facilitytype, dtype: int64

In [46]:
len(lbr)

864

In [17]:
geoms = [Point(xy) for xy in zip(lbr.longitude, lbr.latitude)]
lbr_geo = gpd.GeoDataFrame(lbr, crs='EPSG:4326', geometry=geoms)

In [18]:
lbr_geo

,admin_county,admin_district,today,latitude,longitude,_getcoordinate_altitude,precision,facilitytype,ownership,opendate,...,offer_fpcounsel,offer_adolescentsrhr,offer_sexgender,offer_detectmanage,offer_precenttreat,dup,update_date,geocoding_method,facility,geometry
0,Montserrado,Somalia Drive,2021-08-07,6.331593,-10.743366,4.009424,4.983,clinic,Private (non-faith based),2013,...,1.0,0.0,0.0,0.0,0.0,0,07aug2021,NaN,NaN,POINT (-10.74337 6.33159)
1,Montserrado,Central Monrovia,2021-10-19,6.304997,-10.799709,2.900000,4.978,clinic,"Private, (faith based)",1988,...,NaN,NaN,NaN,NaN,NaN,0,19oct2021,NaN,NaN,POINT (-10.79971 6.30500)
2,Montserrado,St Paul River,2021-11-18,6.412980,-10.778231,-2.712283,4.933,clinic,Private (non-faith based),2018,...,NaN,NaN,NaN,NaN,NaN,0,18nov2021,NaN,NaN,POINT (-10.77823 6.41298)
3,Montserrado,St Paul River,2021-08-23,6.413318,-10.778419,17.595896,4.966,clinic,Public,1950,...,1.0,1.0,0.0,0.0,0.0,0,23aug2021,NaN,NaN,POINT (-10.77842 6.41332)
4,Montserrado,Commonwealth,2021-11-17,6.311818,-10.654602,22.641799,4.950,clinic,"Private, (faith based)",2013,...,0.0,0.0,0.0,0.0,0.0,0,17nov2021,NaN,NaN,POINT (-10.65460 6.31182)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,Grand Cape Mount,Garwula,6/10/2021,6.894938,-11.138828,NaN,NaN,clinic,Public,2007,...,1.0,1.0,1.0,0.0,0.0,0,10-Jun-21,Location from old Master List,Zaway Clinic,POINT (-11.13883 6.89494)
860,Sinoe,Butaw,6/12/2021,5.119427,-9.070022,NaN,NaN,clinic,Public,1974,...,1.0,1.0,1.0,0.0,0.0,1,12-Jun-21,Location from old Master List,Butaw Clinic,POINT (-9.07002 5.11943)
861,Bong,Fuamah,8/8/2021,7.097913,-10.166629,NaN,NaN,clinic,Public,1978,...,1.0,1.0,1.0,0.0,0.0,1,8-Aug-21,Location from old Master List,Degei Clinic,POINT (-10.16663 7.09791)
862,Lofa,Kolahun,9/7/2021,8.221433,-10.173505,NaN,NaN,clinic,Public,1973,...,1.0,1.0,1.0,0.0,0.0,1,7-Sep-21,Location from old Master List,Fangoda Clinic,POINT (-10.17351 8.22143)


### Admin Boundaries

In [24]:
global_admin = '/home/public/Data/GLOBAL/ADMIN/g2015_0_simplified.shp'
adm0 = gpd.read_file(global_admin)
aoi = adm0.loc[adm0.ISO3166_1_==iso3]

In [25]:
global_admin2 = '/home/public/Data/GLOBAL/ADMIN/Admin2_Polys.shp'
adm2 = gpd.read_file(global_admin2)
adm2 = adm2.loc[adm2.ISO3==iso3].copy()
adm2 = adm2.to_crs("EPSG:4326")

### Friction and Population

In [26]:
global_friction_surface = "/home/public/Data/GLOBAL/INFRA/FRICTION_2020/2020_motorized_friction_surface.geotiff"
global_population = "/home/public/Data/GLOBAL/Population/WorldPop_PPP_2020/ppp_2020_1km_Aggregated.tif"

In [27]:
inG = rio.open(global_friction_surface)

In [28]:
# Clip the travel raster to AOI
out_travel_surface = os.path.join(out_folder, "travel_surface.tif")
rMisc.clipRaster(inG, aoi, out_travel_surface, bbox=False, buff=0.1)

/home/wb514197/Repos/gostrocks/src/GOSTRocks/rasterMisc.py:73: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  inD = inD.buffer(buff)


In [29]:
inP = rio.open(global_population)

In [30]:
# Clip the pop raster to AOI
out_pop = os.path.join(out_folder, "WP_2020_1km.tif")
rMisc.clipRaster(inP, aoi, out_pop, bbox=False, buff=0.1)

/home/wb514197/Repos/gostrocks/src/GOSTRocks/rasterMisc.py:73: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  inD = inD.buffer(buff)


In [31]:
travel_surf = rio.open(out_travel_surface)
pop_surf = rio.open(out_pop)

In [32]:
# standardize so that they have the same number of pixels and dimensions
out_pop_surface_std = os.path.join(out_folder, "WP_2020_1km_STD.tif")
rMisc.standardizeInputRasters(pop_surf, travel_surf, out_pop_surface_std, resampling_type="nearest")

[array([[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32),
 {'driver': 'GTiff',
  'dtype': 'float32',
  'nodata': -3.4028234663852886e+38,
  'width': 521,
  'height': 528,
  'count': 1,
  'crs': CRS.from_epsg(4326),
  'transform': Affine(0.008333333333333333, 0.0, -11.599999999999994,
         0.0, -0.008333333333333333, 8.658333333333331)}]

In [33]:
# create a data frame of all points
pop_surf = rio.open(out_pop_surface_std)
pop = pop_surf.read(1, masked=True)
indices = list(np.ndindex(pop.shape))
xys = [Point(pop_surf.xy(ind[0], ind[1])) for ind in indices]
res_df = pd.DataFrame({
    'spatial_index': indices, 
    'xy': xys, 
    'pop': pop.flatten()
})
res_df['pointid'] = res_df.index

/home/wb514197/.conda/envs/graph/lib/python3.7/site-packages/pandas/core/dtypes/cast.py:121: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [34]:
# create MCP object
inG_data = travel_surf.read(1) * 1000 # minutes to travel 1 meter, convert to km
# Correct no data values
inG_data[inG_data < 0] = 9999999999 # untraversable
# inG_data[inG_data < 0] = np.nan
mcp = graph.MCP_Geometric(inG_data)

Separate destinations

In [35]:
lbr_geo_filt = lbr_geo.loc[lbr_geo.intersects(aoi.unary_union)]

In [36]:
lbr_geo_filt.reset_index(inplace=True, drop=True)

In [37]:
len(lbr_geo), len(lbr_geo_filt)

(864, 857)

In [46]:
lbr_geo_filt.facilitytype.value_counts()

clinic           735
health center     87
hospital          24
health post        6
schoold based      4
SDP                1
Name: facilitytype, dtype: int64

In [43]:
hospitals = lbr_geo_filt.loc[lbr_geo_filt.facilitytype=="hospital"].copy()

In [44]:
hospitals.reset_index(inplace=True, drop=True)

In [45]:
len(lbr_geo_filt), len(hospitals)

(857, 24)

In [47]:
res_health = ma.calculate_travel_time(travel_surf, mcp, lbr_geo_filt)[0]
res_hospital = ma.calculate_travel_time(travel_surf, mcp, hospitals)[0]

In [48]:
res_health.mean()

339020541282.76245

In [49]:
res_hospital.mean()

339020541331.23004

In [50]:
res_df.loc[:, f"tt_health"] = res_health.flatten()
res_df.loc[:, f"tt_hospital"] = res_hospital.flatten()

In [51]:
res_df = res_df.loc[res_df['pop']>0].copy()
res_df = res_df.loc[~(res_df['pop'].isna())].copy()

In [52]:
res_df.loc[:,'xy'] = res_df['xy'].apply(Point)
res_gdf = gpd.GeoDataFrame(res_df, geometry='xy', crs='EPSG:4326')

In [53]:
res_gdf.loc[:, 'geometry'] = res_gdf.loc[:, 'xy']

In [54]:
raster_path = out_pop_surface_std

In [55]:
aggregator.rasterize_gdf(res_gdf, 'tt_health', raster_path, os.path.join(out_folder, "tt_health.tif"), nodata=-1)
aggregator.rasterize_gdf(res_gdf, 'tt_hospital', raster_path, os.path.join(out_folder, "tt_hospital.tif"), nodata=-1)